In [ ]:
import sys
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import json
import random
from collections import defaultdict
# เพิ่ม path ของ folder 'src' เพื่อให้ import ไฟล์ของ AST ได้
sys.path.append('./src') 
os.environ['TORCH_HOME'] = '../pretrained_models'  

from models import ASTModel
import dataloader

# ตรวจสอบ Device (ถ้ามี GPU ก็ใช้ GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
def prepare_ast_patient_split_clean(root_path, train_ratio=0.8):
    # 1. จัดกลุ่มไฟล์ตามรหัสผู้ป่วย (Patient ID: 00x)
    patient_groups = {
        "Cough_PTB": defaultdict(list),
        "Cough_Non-PTB": defaultdict(list)
    }
    
    class_map = {"Cough_PTB": "1", "Cough_Non-PTB": "0"}

    for folder_name, label_idx in class_map.items():
        folder_path = os.path.join(root_path, folder_name)
        if not os.path.exists(folder_path): continue
            
        for file in os.listdir(folder_path):
            if file.endswith(".wav"):
                # ดึง Patient ID (00x) จากส่วนหน้าสุดของชื่อไฟล์
                patient_id = file.split('_')[0] 
                full_path = os.path.abspath(os.path.join(folder_path, file))
                patient_groups[folder_name][patient_id].append(full_path)

    train_list, eval_list = [], []
    stats = {
        "train": {"patients": [], "ptb_samples": 0, "non_ptb_samples": 0},
        "eval": {"patients": [], "ptb_samples": 0, "non_ptb_samples": 0}
    }

    # 2. แบ่งข้อมูลรายกลุ่มผู้ป่วย (Patient Level) แยกตามคลาส
    for folder_name, groups in patient_groups.items():
        patient_ids = list(groups.keys())
        random.seed(42) # ล็อก Seed เพื่อให้ผลลัพธ์คงที่
        random.shuffle(patient_ids)
        
        split_idx = int(len(patient_ids) * train_ratio)
        train_ids = patient_ids[:split_idx]
        eval_ids = patient_ids[split_idx:]

        # กลุ่ม Train
        for p_id in train_ids:
            stats["train"]["patients"].append(p_id) # เก็บเฉพาะรหัสผู้ป่วย
            for path in groups[p_id]:
                train_list.append({"wav": path, "labels": class_map[folder_name]})
                if folder_name == "Cough_PTB": stats["train"]["ptb_samples"] += 1
                else: stats["train"]["non_ptb_samples"] += 1
        
        # กลุ่ม Eval
        for p_id in eval_ids:
            stats["eval"]["patients"].append(p_id) # เก็บเฉพาะรหัสผู้ป่วย
            for path in groups[p_id]:
                eval_list.append({"wav": path, "labels": class_map[folder_name]})
                if folder_name == "Cough_PTB": stats["eval"]["ptb_samples"] += 1
                else: stats["eval"]["non_ptb_samples"] += 1

    # 3. บันทึกไฟล์ JSON
    with open('train_data.json', 'w') as f:
        json.dump({"data": train_list}, f, indent=4)
    with open('eval_data.json', 'w') as f:
        json.dump({"data": eval_list}, f, indent=4)

    # 4. แสดงผลสรุปที่สะอาดขึ้น
    print("="*50)
    print("AST DATA PREPARATION SUMMARY")
    print("="*50)
    
    print(f"\n[TRAIN SET]")
    print(f"Total Unique Patients: {len(stats['train']['patients'])} คน")
    print(f"Patient IDs: {', '.join(sorted(stats['train']['patients']))}")
    print(f"Samples: PTB (Class 1) = {stats['train']['ptb_samples']} | Non-PTB (Class 0) = {stats['train']['non_ptb_samples']}")
    
    print(f"\n[EVAL SET]")
    print(f"Total Unique Patients: {len(stats['eval']['patients'])} คน")
    print(f"Patient IDs: {', '.join(sorted(stats['eval']['patients']))}")
    print(f"Samples: PTB (Class 1) = {stats['eval']['ptb_samples']} | Non-PTB (Class 0) = {stats['eval']['non_ptb_samples']}")
    print("="*50)

# เรียกใช้งาน
prepare_ast_patient_split_clean("./Data")

In [ ]:
# ค่า Config สำหรับเสียงไอ 1 วินาทีของคุณ
audio_conf = {
    'num_mel_bins': 128, 
    'target_length': 100,   # 1 วินาที
    'freqm': 0,            # Frequency Masking (ปิดแถบความถี่เพื่อ Augment)
    'timem': 0,            # Time Masking (ลดลงจาก 192 เพราะเสียงเราสั้น)
    'mixup': 0,           # Mixup augmentation
    'dataset': 'audioset',
    'mode': 'train',
    'mean': -3.3831,        # ค่าที่คุณคำนวณได้
    'std': 5.1156,          # ค่าที่คุณคำนวณได้
    'noise': False,
    'skip_norm': False      # ต้องเป็น False เพื่อให้มัน Normalize ข้อมูลให้เรา
}

# สร้าง Dataset
train_dataset = dataloader.AudiosetDataset(
    'train_data.json', 
    label_csv='class_labels_indices.csv', 
    audio_conf=audio_conf
)

# สร้าง DataLoader (ดึงทีละ 4 ไฟล์พอ เพื่อดูตัวอย่าง)
train_loader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=4, 
    shuffle=True, 
    num_workers=0
)

print("Dataset Loaded!")

In [ ]:
# # ดึงข้อมูลมา 1 Batch
# for i, (audio_input, labels) in enumerate(train_loader):
#     # audio_input shape: [batch_size, time_frame, freq_bins]
#     print(f"Input Shape: {audio_input.shape}") 
#     print(f"Labels: {labels}") # จะเห็นเป็น One-hot encoding หรือ Index

#     # ลองวาด Spectrogram ของรูปแรกใน Batch
#     spec = audio_input[0].detach().cpu().numpy()
    
#     # พล็อต Spectrogram
#     plt.figure(figsize=(10, 4))
#     # ต้อง Transpose (.T) เพื่อให้แกน X เป็นเวลา แกน Y เป็นความถี่ (เหมือนรูปทั่วไป)
#     plt.imshow(spec.T, origin='lower', aspect='auto', cmap='inferno')
#     plt.title(f"Spectrogram Input (Normalized & Masked)\nLabel: {labels[0]}")
#     plt.xlabel("Time Frames (Total 100)")
#     plt.ylabel("Mel Frequency Bins (128)")
#     plt.colorbar(format='%+2.0f dB')
#     plt.show()
    
#     break # ดูแค่ Batch เดียวพอ

In [ ]:

import os
import json
import torch
import numpy as np
from PIL import Image
import matplotlib.cm as cm
from tqdm import tqdm

# --- Label → ชื่อ Folder ---
LABEL_TO_FOLDER = {
    "1": "PTB",
    "0": "Non-TB"
}

# --- สร้าง Directory Structure ---
ROOT_DIR = './spectrogram'
for sub in ['ptFile', 'img']:
    for cls in ['PTB', 'Non-TB']:
        os.makedirs(os.path.join(ROOT_DIR, sub, cls), exist_ok=True)

print("Folder structure created:")
print("  spectrogram/ptFile/PTB  ← normalized Kaldi fbank (.pt) สำหรับ model")
print("  spectrogram/ptFile/Non-TB")
print("  spectrogram/img/PTB     ← de-normalized fbank visualized (.png) สำหรับดูรูป")
print("  spectrogram/img/Non-TB\n")

# --- รวมข้อมูลจากทั้ง train และ eval JSON ---
all_audio_list = []
for json_file in ['train_data.json', 'eval_data.json']:
    with open(json_file, 'r') as f:
        data_json = json.load(f)
    all_audio_list.extend(data_json['data'])

print(f"Total files to process: {len(all_audio_list)}")

# --- สร้าง Dataset รวมจากข้อมูลทั้งหมด ---
import tempfile
combined_json_path = os.path.join(tempfile.gettempdir(), 'combined_data.json')
with open(combined_json_path, 'w') as f:
    json.dump({"data": all_audio_list}, f)

combined_dataset = dataloader.AudiosetDataset(
    combined_json_path,
    label_csv='class_labels_indices.csv',
    audio_conf=audio_conf
)

# Dataset mean/std ที่ใช้ normalize (จาก audio_conf)
NORM_MEAN = audio_conf['mean']   # -3.3831
NORM_STD  = audio_conf['std']    # 5.1156

# --- วนลูปสร้างและบันทึก Spectrogram ---
# Spectrogram ที่ได้จาก dataloader คือ Kaldi Mel Filterbank (fbank) ตามที่ README ระบุ:
#   - torchaudio.compliance.kaldi.fbank (htk_compat=True, hanning window)
#   - 128 mel bins, 10ms frame shift, 16kHz
#   - Normalize: (fbank - mean) / (std * 2)  →  ~0 mean, ~0.5 std
print("Starting Spectrogram Extraction...")
for i in tqdm(range(len(combined_dataset))):
    spec, label = combined_dataset[i]   # spec: normalized fbank [time=100, freq=128]

    label_str = all_audio_list[i]['labels']
    class_folder = LABEL_TO_FOLDER.get(label_str, "Non-TB")

    original_wav_path = all_audio_list[i]['wav']
    base_name = os.path.basename(original_wav_path).replace('.wav', '')

    # 1. บันทึก .pt ← normalized fbank ตรงๆ (ค่าที่ model ต้องการตาม README)
    pt_path = os.path.join(ROOT_DIR, 'ptFile', class_folder, base_name + '.pt')
    torch.save(spec, pt_path)

    # 2. บันทึก .png ← de-normalize กลับสู่ค่า log-mel energy จริง แล้ว visualize
    #    de-norm: fbank_original = spec * (std * 2) + mean
    img_path = os.path.join(ROOT_DIR, 'img', class_folder, base_name + '.png')
    fbank_orig = spec.numpy() * (NORM_STD * 2) + NORM_MEAN   # [time, freq] in log-mel dB
    fbank_disp = fbank_orig.T[::-1].copy()                    # [freq, time], low freq at bottom

    # Normalize เพื่อ map สีเท่านั้น (min-max ของ sample นี้)
    vmin, vmax = fbank_disp.min(), fbank_disp.max()
    fbank_norm = (fbank_disp - vmin) / (vmax - vmin + 1e-8)

    rgba = (cm.inferno(fbank_norm) * 255).astype(np.uint8)    # apply inferno colormap
    img = Image.fromarray(rgba[:, :, :3], mode='RGB')
    # resize 8x ให้ดูง่าย (100x128 → 800x1024 px)
    img = img.resize((img.width * 8, img.height * 8), Image.NEAREST)
    img.save(img_path)

print(f"\nDone! Spectrograms saved in '{ROOT_DIR}/'")
print(f"  PTB    : {len([x for x in all_audio_list if x['labels']=='1'])} files")
print(f"  Non-TB : {len([x for x in all_audio_list if x['labels']=='0'])} files")


In [ ]:
achitecture = ASTModel(
            label_dim=2,             # จำนวนคลาส (PTB, NON-PTB)
            fstride=10,              # ต้องเป็น 10 สำหรับ AudioSet pretrain
            tstride=10,              # ต้องเป็น 10 สำหรับ AudioSet pretrain
            input_fdim=128,          # ค่ามาตรฐาน
            input_tdim=100,          # ปรับตามความยาว spectrogram ของคุณ 
            imagenet_pretrain=True,  # เปิดใช้งาน
            audioset_pretrain=True,  # เปิดใช้งาน (AST-P)
            model_size='base384'     # ต้องเป็น base384
        )
ast_mdl = achitecture

In [ ]:

# ===== TB Classification Training with AST-P =====
import argparse
import os
import torch
from traintest import train, validate

# --- 1. DataLoader สำหรับ Train และ Eval ---
# audio_conf สำหรับ Train (เปิด SpecAugment)
train_audio_conf = {
    'num_mel_bins': 128,
    'target_length': 100,
    'freqm': 24,          # mask 24/128 frequency bins (ตาม README แนะนำ ~48 แต่ข้อมูลน้อยใช้ 24)
    'timem': 20,          # mask 20% ของ time frames
    'mixup': 0.0,         # ข้อมูลน้อย ปิด mixup
    'dataset': 'audioset',
    'mode': 'train',
    'mean': -3.3831,
    'std': 5.1156,
    'noise': False,
    'skip_norm': False
}

# audio_conf สำหรับ Eval (ปิด Augmentation ทั้งหมด)
eval_audio_conf = {
    'num_mel_bins': 128,
    'target_length': 100,
    'freqm': 0,
    'timem': 0,
    'mixup': 0.0,
    'dataset': 'audioset',
    'mode': 'evaluation',
    'mean': -3.3831,
    'std': 5.1156,
    'noise': False,
    'skip_norm': False
}

BATCH_SIZE = 8  # ข้อมูลน้อย ใช้ batch เล็ก

train_loader = torch.utils.data.DataLoader(
    dataloader.AudiosetDataset('train_data.json', label_csv='class_labels_indices.csv', audio_conf=train_audio_conf),
    batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True
)

eval_loader = torch.utils.data.DataLoader(
    dataloader.AudiosetDataset('eval_data.json', label_csv='class_labels_indices.csv', audio_conf=eval_audio_conf),
    batch_size=BATCH_SIZE * 2, shuffle=False, num_workers=0, pin_memory=True
)

print(f"Train samples: {len(train_loader.dataset)}")
print(f"Eval  samples: {len(eval_loader.dataset)}")


In [ ]:

# --- 2. สร้าง AST-P Model (AudioSet Pretrained) ---
ast_model = ASTModel(
    label_dim=2,              # PTB vs Non-TB
    fstride=10,
    tstride=10,
    input_fdim=128,
    input_tdim=100,
    imagenet_pretrain=True,
    audioset_pretrain=True,   # AST-P: ใช้ AudioSet pretrained weights
    model_size='base384'
)


In [ ]:

# --- 3. ตั้งค่า Training Arguments ---
args = argparse.Namespace(
    # Experiment
    exp_dir        = './exp/tb_ast_p',

    # Data
    dataset        = 'audioset',

    # Model
    n_class        = 2,

    # Training
    lr             = 1e-5,      # AST ต้องการ lr เล็กมาก (README แนะนำ 10x เล็กกว่า CNN)
    n_epochs       = 30,
    batch_size     = BATCH_SIZE,
    n_print_steps  = 10,
    save_model     = True,

    # Loss & Metrics
    # CE + acc เหมาะกับ binary classification (2 class, single-label)
    loss           = 'CE',
    metrics        = 'acc',

    # Learning Rate Scheduler (MultiStepLR)
    # เริ่ม decay ที่ epoch 10, ทุก 5 epoch, decay rate 0.5
    lrscheduler_start = 10,
    lrscheduler_step  = 5,
    lrscheduler_decay = 0.5,

    # Warmup
    warmup         = True,
)

os.makedirs(f'{args.exp_dir}/models', exist_ok=True)
print(f"Experiment will be saved to: {args.exp_dir}")
print(f"Training config: lr={args.lr}, epochs={args.n_epochs}, loss={args.loss}, metrics={args.metrics}")


In [ ]:
# --- 4. รัน Training ---
train(ast_model, train_loader, eval_loader, args)

best_model_path = f'{args.exp_dir}/models/best_audio_model.pth'
print(f"\nTraining complete. Best model saved to: {best_model_path}")

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# ตรวจสอบว่าไฟล์โมเดลมีอยู่หรือไม่
if not os.path.exists(best_model_path):
    raise FileNotFoundError(f"Model file not found: {best_model_path}. Ensure training is completed and the model is saved.")

# โหลด best model
best_model = ASTModel(label_dim=2, fstride=10, tstride=10,
                      input_fdim=128, input_tdim=100,
                      imagenet_pretrain=False, audioset_pretrain=False,
                      model_size='base384', verbose=False)
best_model = torch.nn.DataParallel(best_model)
best_model.load_state_dict(torch.load(best_model_path, map_location=device))
print(f"Loaded best model from: {best_model_path}")

# รัน validate บน eval set
stats, eval_loss = validate(best_model, eval_loader, args, epoch='best')
acc  = stats[0]['acc']
auc  = stats[0]['auc']
print(f"\n===== Best Model Evaluation =====")
print(f"Accuracy : {acc:.4f} ({acc*100:.2f}%)")
print(f"AUC      : {auc:.4f}")
print(f"Eval Loss: {eval_loss:.4f}")

# พล็อต training curve จาก result.csv
result_path = f'{args.exp_dir}/result.csv'
if os.path.exists(result_path):
    cols = ['acc/mAP', 'mAUC', 'avg_precision', 'avg_recall', 'd_prime',
            'train_loss', 'valid_loss', 'cum_acc/mAP', 'cum_mAUC', 'lr']
    df = pd.read_csv(result_path, header=None, names=cols)
    df = df[df['acc/mAP'] != 0]  # ตัด epoch ที่ยังไม่ได้รัน

    fig, axes = plt.subplots(1, 3, figsize=(15, 4))

    axes[0].plot(df['acc/mAP'], marker='o', label='Acc')
    axes[0].plot(df['cum_acc/mAP'], marker='s', linestyle='--', label='Cum Acc')
    axes[0].set_title('Accuracy per Epoch')
    axes[0].set_xlabel('Epoch')
    axes[0].legend()
    axes[0].grid(True)

    axes[1].plot(df['train_loss'], marker='o', label='Train Loss')
    axes[1].plot(df['valid_loss'], marker='s', linestyle='--', label='Eval Loss')
    axes[1].set_title('Loss per Epoch')
    axes[1].set_xlabel('Epoch')
    axes[1].legend()
    axes[1].grid(True)

    axes[2].plot(df['mAUC'], marker='o', label='AUC')
    axes[2].set_title('AUC per Epoch')
    axes[2].set_xlabel('Epoch')
    axes[2].legend()
    axes[2].grid(True)

    plt.tight_layout()
    plt.show()